### Data Source Integration

##### Price Data — Coingecko API

In [6]:
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os
import joblib

# === LOAD .env FILE ===
load_dotenv()
api_key = os.getenv("GECKO_KEY")

if not api_key:
    raise ValueError("API_KEY not found in .env file")

# === FUNCTION TO FETCH & SAVE DATA ===
def fetch_and_save_coingecko_data(coin_id="ethereum", vs_currency="usd", days="30", interval="daily"):
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    headers = {
        "accept": "application/json",
        "x-cg-pro-api-key": api_key
    }
    params = {
        "vs_currency": vs_currency,
        "days": days,
        "interval": interval
    }

    # Fetch data
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Error {response.status_code}: {response.text}")

    data = response.json()
    prices = data.get("prices", [])
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)

    # Create dynamic filename
    filename = f"{coin_id}_{vs_currency}_{days}d_{interval}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.joblib"
    filepath = os.path.join("data", filename)

    # Ensure folder exists
    os.makedirs("data", exist_ok=True)

    # Save file
    joblib.dump(df, filepath)
    print(f"Data saved to {filepath}")

    return filepath

# === FUNCTION TO LOAD SAVED DATA ===
def load_saved_data(filepath):
    return joblib.load(filepath)

# ==== EXAMPLE USAGE ====
# Save new data
file_path = fetch_and_save_coingecko_data(coin_id="ethereum", days="30", interval="daily")

# Load saved data later
df_loaded = load_saved_data(file_path)
print(df_loaded.head())


Data saved to data\ethereum_usd_30d_daily_20250811_114953.joblib
                  price
timestamp              
2025-07-13  2942.961606
2025-07-14  2974.265495
2025-07-15  3012.179707
2025-07-16  3133.069858
2025-07-17  3368.126522


##### Whale Activity — Dune Sim API 

In [7]:
#Extract on-chain large transfers > $100K for the same token and timeframe (e.g., Ethereum, past 7 days).

import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

# =========================
# Configurable Parameters
# =========================
QUERY_ID = 5601937  # Dune query ID for whale transfers
OUTPUT_DIR = "data/dune_whale_data"  # where CSV files will be stored
SAVE_FILENAME = f"whale_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

# =========================
# Setup
# =========================
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_KEY")  # Or use "DUNE_API_KEY" in your .env

if not DUNE_API_KEY:
    raise ValueError("DUNE_KEY not found in .env file")

os.makedirs(OUTPUT_DIR, exist_ok=True)

headers = {
    "x-dune-api-key": DUNE_API_KEY,
    "accept": "application/json"
}

# =========================
# Helper Function to Run Query
# =========================
def fetch_dune_query(query_id):
    """Run a Dune query and return results as a DataFrame."""
    # Trigger execution
    execution_url = f"https://api.dune.com/api/v1/query/{query_id}/execute"
    exec_response = requests.post(execution_url, headers=headers)
    exec_response.raise_for_status()
    execution_id = exec_response.json()["execution_id"]

    # Poll for results
    result_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results"
    while True:
        res = requests.get(result_url, headers=headers)
        res.raise_for_status()
        data = res.json()
        state = data.get("state")

        if state == "QUERY_STATE_COMPLETED":
            break
        elif state in ["QUERY_STATE_FAILED", "QUERY_STATE_CANCELLED"]:
            raise RuntimeError(f"Query failed or was cancelled: {data}")
        else:
            print("Waiting for results...")
            time.sleep(5)

    results = data["result"]["rows"]
    return pd.DataFrame(results)

# =========================
# Main Script
# =========================
if __name__ == "__main__":
    print(f"Fetching whale data from Dune query {QUERY_ID}...")
    df_whales = fetch_dune_query(QUERY_ID)

    # Save to CSV
    output_path = os.path.join(OUTPUT_DIR, SAVE_FILENAME)
    df_whales.to_csv(output_path, index=False)
    print(f"✅ Data saved to: {output_path}")

    # Preview
    print(df_whales.head())


Fetching whale data from Dune query 5601937...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting for results...
Waiting fo